실습평가
===
평가아아아아아아~~~

In [1]:
ARRIVAL = 'arrival'
WAITING = 'waiting'
EATING = 'eating'
WORKING = 'working'
LEAVING = 'leaving'

#%%
# define classes here
class Person:
    def __init__(self, person_id):
        self.person_id = person_id
        self.waiting_time = 0
        self.state = None
        
#%%        
class Visitor(Person):
    def __init__(self, arrival_time):
        super().__init__(arrival_time)
        self.state = ARRIVAL
        self.remaining_eating_time = 0
        
    def order(self):
        self.state = WAITING
        
    def eat(self):
        self.state = EATING
        self.remaining_eating_time = 20
        
    def leave(self):
        self.state = LEAVING
        
    def after_one_minute(self):
        if self.state == EATING:
            if self.remaining_eating_time > 0:
                self.remaining_eating_time -= 1
            else:
                self.leave()
        if self.state == ARRIVAL or self.state == WAITING:
            self.waiting_time += 1
            
        return self
    
    def __str__(self):
        visitor_str = "visitor arrived at {}, {}".format(self.person_id, self.state)
        return visitor_str
        
            
#%%
class Employee(Person):
    def __init__(self, person_id):
        super().__init__(person_id)
        self.state = WAITING
        self.remaining_cooking_time = 0
        self.visitor = None
        
    def cook(self, visitor):
        visitor.order()
        self.state = WORKING
        self.remaining_cooking_time = 15
        self.visitor = visitor
        
    def serve(self):
        self.visitor.eat()
        self.state = WAITING
        self.visitor = None
        
    def after_one_minute(self):
        if self.state == WORKING:
            if self.remaining_cooking_time > 0:
                self.remaining_cooking_time -= 1
            else:
                self.serve()
        else:
            self.waiting_time += 1
            
        return self
    
    def __str__(self):
        employee_str = "employee id {}, {}".format(self.person_id, self.state)
        return employee_str

#%%

In [2]:
ARRIVAL_TIMES = [1, 3, 5, 10, 15, 18, 25, 30, 40, 52, 55, 60, 70, 75, 80, 85, 94, 98]

#%%
# define a class here
class Restaurant:
    def __init__(self, num_seats, num_employees, closing_time):
        self.num_seats = num_seats
        self.closing_time = closing_time
        self.current_time = 0
        self.visitors = []
        self.total_visitors_served = []
        self.employees = [Employee(emp_id+1) for emp_id in range(num_employees)]
        
    def visitor_arrived(self, visitor_obj):
        if len(self.visitors) == self.num_seats:
            self.visitors.append(visitor_obj)
            self.total_visitors_served.append(visitor_obj)
        else:
            visitor_obj.leave()
        
    def order_process(self):
        avail_employee = []
        arriv_visitor = []
        
        for employee in self.employees:
            if employee.state == WAITING:
                avail_employee.append(employee)
                
        for visitor in self.visitors:
            if visitor.state == ARRIVAL:
                arriv_visitor.append(visitor)
        
        if avail_employee and arriv_visitor:
            avail_employee[0].cook(arriv_visitor[0])
            return self.order_process()
                        
    def after_one_minute(self):
        for visitor in self.visitors:
            visitor.after_one_minute()
            if visitor.state == LEAVING:
                self.visitors.remove(visitor)
                
        for employee in self.employees:
            employee.after_one_minute()
            
    def operate(self):
        self.current_time += 1
        if self.current_time == self.closing_time:
            self.summarize_result()
        else:    
            if self.current_time in ARRIVAL_TIMES:
                visitor_obj = Visitor(self.current_time)
                self.visitor_arrived(visitor_obj)
                
            self.order_process()
            self.after_one_minute()
            return self.operate()
        
    def summarize_result(self):
        num_served = len(self.total_visitors_served)
        num_not_served = len(ARRIVAL_TIMES) - num_served
        
        visitor_waiting_avg = 0
        for visitor in self.total_visitors_served:
            visitor_waiting_avg += visitor.waiting_time
        visitor_waiting_avg /= num_served
        
        employee_waiting_avg = 0
        for employee in self.employees:
            employee_waiting_avg += employee.waiting_time
        employee_waiting_avg /= len(self.employees)
        employee_working_avg = self.closing_time - employee_waiting_avg
        
        print("served: {}, not_served: {}".format(num_served, num_not_served))
        print("visitors average waiting times: {}".format(visitor_waiting_avg))
        print("employees average waiting times: {}".format(employee_waiting_avg))
        print("employees average working times: {}".format(employee_working_avg))



#%%
if __name__ == '__main__':
    rest = Restaurant(5, 2, 100)
    rest.operate()

#%%

ZeroDivisionError: division by zero